In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import shap
import numpy as np

In [14]:
# 저장한 모델 경로
model_path = 'models/models0506_klue/'

# 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [15]:
# 임의의 문자열
text = "이 영화 정말 재미있었어요!"

In [16]:
# 입력 데이터 토큰화
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

In [17]:
# 모델 추론
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

In [18]:
# 결과 출력
predicted_class = torch.argmax(predictions, dim=1).item()
confidence = predictions[0][predicted_class].item()
print(f"Predicted class: {predicted_class}, Confidence: {confidence:.4f}")

Predicted class: 1, Confidence: 0.5373


In [25]:
# SHAP용 모델 설명 함수 정의
def forward_func(inputs):
    outputs = model(**inputs)
    return torch.nn.functional.softmax(outputs.logits, dim=-1).detach().numpy()

# SHAP explainer 생성
explainer = shap.Explainer(forward_func, tokenizer)

# 임의의 문자열을 토큰화하여 SHAP 입력 생성
shap_inputs = tokenizer([text], return_tensors="pt", truncation=True, padding=True)

# SHAP 값 계산
shap_values = explainer(shap_inputs)

# 시각화
shap.initjs()
shap.summary_plot(shap_values[0], tokenizer.convert_ids_to_tokens(shap_inputs['input_ids'][0]))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()